In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import roc_curve

pd.options.display.max_columns = None
pd.options.display.max_rows = 100

# Applying the PD Model

In [2]:
summary_table = pd.read_csv('../processed/05_summary_table.csv')
summary_table.head()

,Feature name,Coefficients,p_values
0,Intercept,1.483647,NaN
1,grade_A,0.889716,1.387834e-18
2,grade_B,0.761152,1.163630e-31
3,grade_C,0.579016,8.214773e-22
4,grade_D,0.426654,1.107295e-13


In [3]:
ref_categories = ['grade:G',
'home_ownership:RENT_OTHER_NONE_ANY',
'addr_state:ND_NE_IA_NV_FL_HI_AL',
'verification_status:Verified',
'purpose:educ__sm_b__wedd__ren_en__mov__house',
'initial_list_status:f',
'term:60',
'emp_length:0',
'mths_since_issue_d:>84',
'int_rate:>20.281',
'mths_since_earliest_cr_line:<140',
'inq_last_6mths:>6',
'acc_now_delinq:0',
'annual_inc:<20K',
'dti:>35',
'mths_since_last_delinq:0-3',
'mths_since_last_record:0-2']

### Creating a Scorecard

In [4]:
df_ref_categories = pd.DataFrame(ref_categories, columns = ['Feature name'])
df_ref_categories['Coefficients'] = 0
df_ref_categories['p_values'] = np.nan
df_ref_categories

,Feature name,Coefficients,p_values
0,grade:G,0,NaN
1,home_ownership:RENT_OTHER_NONE_ANY,0,NaN
2,addr_state:ND_NE_IA_NV_FL_HI_AL,0,NaN
3,verification_status:Verified,0,NaN
4,purpose:educ__sm_b__wedd__ren_en__mov__house,0,NaN
5,initial_list_status:f,0,NaN
6,term:60,0,NaN
7,emp_length:0,0,NaN
8,mths_since_issue_d:>84,0,NaN
9,int_rate:>20.281,0,NaN


In [5]:
df_scorecard = pd.concat([summary_table, df_ref_categories])
df_scorecard = df_scorecard.sort_values('Feature name').reset_index()

In [6]:
def feature_name(row):
    if(row.startswith('acc_now_delinq')): return 'acc_now_delinq'
    if(row.startswith('grade')): return 'grade'
    if(row.startswith('addr_state')): return 'addr_state'
    if(row.startswith('annual_inc')): return 'annual_inc'
    if(row.startswith('dti')): return 'dti'
    if(row.startswith('emp_length')): return 'emp_length'
    if(row.startswith('home_ownership')): return 'home_ownership'
    if(row.startswith('initial_list_status')): return 'initial_list_status'
    if(row.startswith('inq_last')): return 'inq_last'
    if(row.startswith('int_rate')): return 'int_rate'
    if(row.startswith('mths_since_earliest_cr_line')): return 'mths_since_earliest_cr_line'
    if(row.startswith('mths_since_last_record')): return 'mths_since_last_record'
    if(row.startswith('purpose')): return 'purpose'
    if(row.startswith('term')): return 'term'
    if(row.startswith('verification_status')): return 'verification_status'
    if(row.startswith('mths_since_issue_d')): return 'mths_since_issue_d'
    if(row.startswith('mths_since_last_delinq')): return 'mths_since_last_delinq'
    return row

In [7]:
df_scorecard['Original feature name'] = df_scorecard['Feature name'].apply(feature_name)
df_scorecard['Original feature name'].unique()

array(['Intercept', 'acc_now_delinq', 'addr_state', 'annual_inc', 'dti',
       'emp_length', 'grade', 'home_ownership', 'initial_list_status',
       'inq_last', 'int_rate', 'mths_since_earliest_cr_line',
       'mths_since_issue_d', 'mths_since_last_delinq',
       'mths_since_last_record', 'purpose', 'term', 'verification_status'],
      dtype=object)

In [8]:
min_score = 300
max_score = 850

In [9]:
df_scorecard.groupby('Original feature name')['Coefficients'].min()

Original feature name
Intercept                      1.483647
acc_now_delinq                 0.000000
addr_state                     0.000000
annual_inc                    -0.032055
dti                            0.000000
emp_length                     0.000000
grade                          0.000000
home_ownership                 0.000000
initial_list_status            0.000000
inq_last                      -1.286869
int_rate                       0.000000
mths_since_earliest_cr_line    0.000000
mths_since_issue_d            -0.308981
mths_since_last_delinq         0.000000
mths_since_last_record        -0.492287
purpose                        0.000000
term                           0.000000
verification_status           -0.001729
Name: Coefficients, dtype: float64

In [10]:
min_sum_coef = df_scorecard.groupby('Original feature name')['Coefficients'].min().sum()
min_sum_coef

-0.6382741431187717

In [11]:
max_sum_coef = df_scorecard.groupby('Original feature name')['Coefficients'].max().sum()
max_sum_coef

7.042439238384943

In [12]:
df_scorecard['Score - Calculation'] = df_scorecard['Coefficients'] * (max_score - min_score) / (max_sum_coef - min_sum_coef)

In [13]:
df_scorecard.loc[0,'Score - Calculation'] = ((df_scorecard['Coefficients'][0] - min_sum_coef) / (max_sum_coef - min_sum_coef)) * (max_score - min_score) + min_score

In [14]:
df_scorecard['Score - Preliminary'] = df_scorecard['Score - Calculation'].round()

In [15]:
min_sum_score_prel = df_scorecard.groupby('Original feature name')['Score - Preliminary'].min().sum()
min_sum_score_prel

301.0

In [16]:
max_sum_score_prel = df_scorecard.groupby('Original feature name')['Score - Preliminary'].max().sum()
max_sum_score_prel

850.0

In [17]:
df_scorecard['Score - Final'] = df_scorecard['Score - Preliminary']
df_scorecard.loc[0,'Score - Final'] = 451

In [18]:
df_scorecard.loc[46,'Score - Final'] = 65

In [19]:
df_scorecard.groupby('Original feature name')['Score - Final'].min().sum(), df_scorecard.groupby('Original feature name')['Score - Final'].max().sum()

(300.0, 850.0)

# Score prediction

In [20]:
df_scorecard.head(1)

,index,Feature name,Coefficients,p_values,Original feature name,Score - Calculation,Score - Preliminary,Score - Final
0,0,Intercept,1.483647,NaN,Intercept,451.946395,452.0,451.0


In [21]:
x_test = pd.read_csv('../processed/x_test_04_output.csv')
x_test.insert(0, 'Intercept', 1)

In [22]:
x_test.head(1)

,Intercept,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,emp_length_int,earliest_cr_line_proc,earliest_cr_line_months,term_int,issue_d_date,mths_since_issue_d,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,sub_grade_A1,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,loan_status_Charged Off,loan_status_Current,loan_status_Default,loan_status_Does not meet the credit policy. Status:Charged Off,loan_status_Does not meet the credit policy. Status:Fully Paid,loan_status_Fully Paid,loan_status_In Grace Period,loan_status_Late (16-30 days),loan_status_Late (31-120 days),purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,addr_state_AK,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,initial_list_status_f,initial_list_status_w,home_ownership_OTHER_NONE_ANY_RENT,addr_state_ND,addr_state_ND_NE_IA_NV_FL_HI_AL,addr_state_NM_VA,addr_state_OK_TN_MO_LA_MD_NC,addr_state_UT_KY_AZ_NJ,addr_state_AR_MI_PA_OH_MN,addr_state_RI_MA_DE_SD_IN,addr_state_GA_WA_OR,addr_state_WI_MT,addr_state_IL_CT,addr_state_KS_SC_CO_VT_AK_MS,addr_state_WV_NH_WY_DC_ME_ID,purpose_educational_renewable_energy_other_house,purpose_medical_wedding_vacation,purpose_major_purchase_car_home_improvement,term_int_36,term_int_60,emp_length_int_0,emp_length_int_1,emp_length_int_2_4,emp_length_int_5_6,emp_length_int_7_9,emp_length_int_10,mths_since_issue_d_27,mths_since_issue_d_27_29,mths_since_issue_d_29_30,mths_since_issue_d_30_39,mths_since_issue_d_39_45,mths_since_issue_d_45_56,mths_since_issue_d_56_84,mths_since_issue_d_84,int_rate_9.548,int_rate_9.548-12.025,int_rate_12.025-15.74,int_rate_15.74-20.281,int_rate_>20.281,delinq_2yrs:0,d

In [23]:
inputs_test_with_ref_cat_w_intercept = x_test[df_scorecard[df_scorecard['Score - Final']!=0]['Feature name'].values]

In [24]:
scorecard_scores = df_scorecard[df_scorecard['Score - Final']!=0][['Score - Final']]

In [25]:
y_scores = inputs_test_with_ref_cat_w_intercept.dot(scorecard_scores.values)

# Cutpoints

In [26]:
sum_coef_from_score = ((y_scores - min_score) / (max_score - min_score)) * (max_sum_coef - min_sum_coef) + min_sum_coef

In [27]:
y_hat_proba_from_score = np.exp(sum_coef_from_score) / (np.exp(sum_coef_from_score) + 1)
y_hat_proba_from_score.head(5)

,0
0,0.933608
1,0.908409
2,0.908409
3,0.955909
4,0.964423


In [28]:
y_test = pd.read_csv('../processed/06_predictions.csv')
y_test = y_test.sort_values('index')
y_test.head()

,index,Unnamed: 0,pred_1,pred_2,pred_2_tr,y_test,obs,non_defaults,defaults,obs_pct,non_defaults_pct,defaults_pct
56000,0,0,0.934371,0.934902,1,1,56000,48512,7488,0.600491,0.574937,0.843433
40229,1,1,0.902951,0.909765,1,1,40229,33965,6264,0.431378,0.402534,0.705564
39948,2,2,0.906280,0.909248,1,1,39948,33705,6243,0.428365,0.399452,0.703199
71877,3,3,0.955018,0.956573,1,1,71877,63523,8354,0.770741,0.752838,0.940978
77336,4,4,0.964563,0.964068,1,1,77336,68785,8551,0.829278,0.815201,0.963167


In [29]:
fpr, tpr, thresholds = roc_curve(y_test.y_test,y_test.pred_2)
df_cutoffs = pd.concat([pd.DataFrame(thresholds), pd.DataFrame(fpr), pd.DataFrame(tpr)], axis = 1)
df_cutoffs.columns = ['thresholds', 'fpr', 'tpr']
df_cutoffs.loc[0,'thresholds'] = 1 - 1/np.power(10,16)

df_cutoffs['Score'] = ((np.log(df_cutoffs['thresholds'] / (1 - df_cutoffs['thresholds'])) - min_sum_coef) * ((max_score - min_score) / (max_sum_coef - min_sum_coef)) + min_score).round()
df_cutoffs.loc[0,'Score'] = max_score

df_cutoffs.head()
df_cutoffs.tail()

,thresholds,fpr,tpr,Score
15697,0.542286,0.999775,0.999917,358.0
15698,0.540949,0.999775,0.999929,357.0
15699,0.540892,0.999887,0.999929,357.0
15700,0.513890,0.999887,1.000000,350.0
15701,0.512922,1.000000,1.000000,349.0


In [30]:
def n_approved(p):
    return np.where(y_test['pred_2'] >= p, 1, 0).sum()

df_cutoffs['N Approved'] = df_cutoffs['thresholds'].apply(n_approved)
df_cutoffs['N Rejected'] = y_test['pred_2'].shape[0] - df_cutoffs['N Approved']
df_cutoffs['Approval Rate'] = df_cutoffs['N Approved'] / y_test['pred_2'].shape[0]
df_cutoffs['Rejection Rate'] = 1 - df_cutoffs['Approval Rate']

In [31]:
df_cutoffs.loc[2000:2020]

,thresholds,fpr,tpr,Score,N Approved,N Rejected,Approval Rate,Rejection Rate
2000,0.944588,0.106093,0.347579,549.0,30270,62987,0.324587,0.675413
2001,0.944587,0.106093,0.347591,549.0,30271,62986,0.324598,0.675402
2002,0.944586,0.106206,0.347591,549.0,30272,62985,0.324608,0.675392
2003,0.944577,0.106206,0.347685,549.0,30280,62977,0.324694,0.675306
2004,0.944576,0.106431,0.347685,549.0,30282,62975,0.324716,0.675284
2005,0.944468,0.106431,0.348562,549.0,30356,62901,0.325509,0.674491
2006,0.944464,0.106544,0.348562,549.0,30357,62900,0.325520,0.674480
2007,0.944436,0.106544,0.348788,549.0,30376,62881,0.325724,0.674276
2008,0.944436,0.106656,0.348788,549.0,30377,62880,0.325734,0.674266
2009,0.944414,0.106656,0.348989,549.0,30394,62863,0.325917,0.674083


In [32]:
df_cutoffs.to_csv('../processed/07_cutoffs.csv',index=False)
df_scorecard.to_csv('../processed/07_scorecard.csv',index=False)